In [16]:
import warnings
#!pip install --upgrade pyarrow
#!pip install lightgbm#
#!pip install statsmodels
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.ensemble import GradientBoostingClassifier

# Agregar la carpeta 'src' al sistema de rutas
sys.path.append(os.path.abspath('../src'))
# Ahora puedes importar el archivo o los módulos
from funciones import *
#import scipy.stats as stats
#import lightgbm as lgb
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.preprocessing import LabelEncoder
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans
import statsmodels.api as sm

pd.set_option('display.max_columns', None)  # Esto mostrará todas las columnas
pd.set_option('display.max_rows', None)  # Esto mostrará todas las filas

In [2]:
dict_path=r'../src/data_dict.xlsx'
dict = pd.read_excel(dict_path)
dict.head()

,Factores,NUMERICA,ORDINAL,CONTINUA,DISCRETA,LABEL,CATEGORÍA,MISSING,format
0,AGRUPAGENTE,1,0.0,0,1.0,Agrupacion del agente,03.Agente,True,FMT_AGRUPAGENTE
1,Altitud,1,1.0,0,1.0,Altitud (m),06.Geográfica,True,FMT_Altitud
2,antigedif,1,1.0,0,1.0,Antiguedad del edificio,04.Riesgo,True,FMT_antigedif
3,antigref,1,1.0,0,1.0,Antiguedad de la reforma,04.Riesgo,True,FMT_antigref
4,ANTIGUEDAD_VIVIENDA,1,1.0,0,1.0,Antiguedad de la vivienda (new),04.Riesgo,True,FMT_ANTIGUEDAD_VIVIENDA


In [3]:

df_path=r'../src/AGUA.parquet'
df_muestra=pd.read_parquet(df_path)
df_muestra.head()

,AGRUPAGENTE,Altitud,antigedif,antigref,ANTIGUEDAD_VIVIENDA,anualidad_reemplazo,anualidad_seguro,AnyoCons_edif,AnyoCons_edif_total,AUX,bloqueo,CANAL,catage_emp,catage_glob,catage_hog,catage_tur,CATTAG,cl_cobro,codigo_grp,CODINE,CODPOST,CODPROV,COMESP,CONFCOMELE,Contr_Agr_pct,Contr_Cons_pct,Contr_ConvIndefH_pct,Contr_ConvIndefm_pct,Contr_Ind_pct,Contr_IndefH_pct,Contr_IndefM_pct,Contr_Serv_pct,Contr_TempH_pct,Contr_Tempm_pct,Contr_tot_pct,Conv_AVG,CORINE_VARIETY,CUPD_CAP_Corr_aguaacagbc,CUPD_Exc_aguaacagbc,DC,densidad_cpost,DIAS_PREC_SUP_10MM,DIAS_PREC_SUP_1MM,DIAS_PREC_SUP_30MM,DIAS_VIENT_MAX_60KMH,DIAS_VIENT_MAX_70KMH,DIAS_VIENT_MAX_80KMH,DIAS_VIENT_MED_30KMH,DIAS_VIENT_MED_40KMH,DIAS_VIENT_MED_50KMH,DIST_COAST,Duracion_per_calido,Duracion_per_seco,edadtom,Elevation_AVG,EMSU_RISK,EQ_AVG,EQ_RISK,ERA5_Daily_avg_u,Etiq_AGUAACAGBC,ETP_anual,EU_IMP_IBU_100m,EU_IMP_IBU_10m,EU_IMP_IMD_100m,EU_IMP_IMD_10m,EU_LC_L1,EU_LC_L2,EU_TOPO_ASP_D4,EU_TOPO_ASP_D8,EU_TOPO_CONV,EU_TOPO_DIST_COAST,EU_TOPO_ELE,EU_TOPO_HAND,EU_TOPO_SLO,EU_VEG_FTY_100m,EU_VEG_FTY_10m,EU_VEG_TCD_100m,EU_VEG_TCD_10m,exp_corr_ACAGBC,EXPOSICION,extension,Factor_R,FI_RISK,Flood_AVG,FLRF_RISK,FLSW_RISK,FP,GMSeguros_indice_med,HA_RISK,HABITANTES,Hail_AVG,IMP_GMIS,Imperv_AVG,Ind_Turc_Regadio,Ind_Turc_Secano,K_ACAGBC,Landslide_AVG,LC_100m,LC_10m,MediadorLMI,mediaEdad,NIF_TIPO,NP_RB,nse,num_declaraciones,num_habitantes,p_africa,p_agpA,p_agpB,p_ALEM,p_americana,p_arabian,P_ARABIAN_VS_TOT,p_consF,p_cpers,p_dpers,p_east_EUR,P_EAST_EUR_VS_TOT,p_espa,p_extra,P_EXTRA_RESTO,p_induC,p_induD,p_induE,p_mcpers,p_p0004,p_p0019,p_p0509,p_p100M,p_p1014,p_p1519,p_p2024,p_p2039,p_p2529,p_p3034,p_p3539,p_p4044,p_p4064,p_p4549,p_p5054,p_p5559,p_p6064,p_p6569,p_p65M,p_p7074,p_p7579,p_p8084,p_p8589,p_p9094,p_p9599,p_portales_A,p_portales_B,p_portales_C,p_portales_D,p_portales_E,p_portales_F,p_portales_NA,p_servG,p_servH,p_servI,p_servJ,p_servK,p_servL,p_servM,p_servN,p_servO,p_servP,p_servQ,P_SUDAMER_VS_TOT,p_tpers,p_UE27,p_upers,p_WEST_EUR,P_WEST_EUR_VS_TOT,PACRAS,Paro_0024_pct,Paro_2544_pct,Paro_Agr_pct,Paro_Cons_pct,Paro_Dese_pct,Paro_Ind_pct,Paro_tot_pct,PCT_CLIENTE_AP,PCT_DTO_COMERCIAL,PCT_DTO_COMISION_AGENTE,PCT_DTO_TRASPASO,pct1524Tot,pct2544Tot,pctActTot,pctJovenTot,pctJubiladoTot,pctM45Tot,Pluviometria_anual,pob_1524Tot,pob_2544Tot,pob_actTot,pob_JovenTot,pob_JubiladoTot,pob_M45Tot,pob_Total,POL_ACCIDENTES,POL_AHORRO,POL_AUTOS,POL_COL_AHORRO,POL_COL_PENSIONES,POL_COL_RIESGO,POL_COMERCIO,POL_COMUNIDADES,POL_EMPRESAS,POL_FLOTAS,POL_HOGAR,POL_IND_AHORRO,POL_IND_FINANCIERO,POL_IND_PENSIONES,POL_IND_RIESGO,POL_MOTOS,POL_NO_VIDA,POL_NOAUTOS,POL_PATRIMONIALES,POL_PENSIONES,POL_RIESGO,POL_SALUD,POL_TOTAL,POL_TURISMOS,POL_VEH_COMERCIAL,POL_VIDA,POLAGE_ACCIDENTES,POLAGE_AUTOS,POLAGE_COMERCIO,POLAGE_COMUNIDADES,POLAGE_FLOTAS,POLAGE_HOGAR,POLAGE_MOTOS,POLAGE_NO_VIDA,POLAGE_NOAUTOS,POLAGE_TURISMOS,POLAGE_VEH_COMERCIAL,POP_GHS2015,POP_GPW2020,Population_AVG,PREC_E_OBS_100Y,PREC_E_OBS_10Y,PREC_E_OBS_25Y,PREC_E_OBS_50Y,PREC_E_OBS_5Y,PREC_ECAD_100Y,PREC_ECAD_10Y,PREC_ECAD_25Y,PREC_ECAD_50Y,PREC_ECAD_5Y,PREC_ERA5_100Y,PREC_ERA5_10Y,PREC_ERA5_25Y,PREC_ERA5_50Y,PREC_ERA5_5Y,PROPIETARIO,RAMO,ratDecl_Conjuntas,ratDecl_xHab,ratTitulares_xHab,renta_bruta,renta_bruta_media,renta_disponible,ROBINSON,SC_RISK,Slope_AVG,stro_1a_AGUAACAGBC,stro_2a_AGUAACAGBC,stro_3a_AGUAACAGBC,stro_4a_AGUAACAGBC,stro_5a_AGUAACAGBC,stro_Corr_AGUAACAGBC,Stro_G_aguaacagbc,SUPERFICIE,TAG,TC_RISK,TEMP_MAX_10Y,TEMP_MAX_30Y,TEMP_MAX_50Y,TEMP_MIN_10Y,TEMP_MIN_30Y,TEMP_MIN_50Y,Temperatura_med_anual,Temperatura_med_max_mes_calido,Temperatura_med_min_mes_frio,tipoImpositivoMedio,titulares,Tornado_AVG,tot_pob,Tree_cover_AVG,VEG_TCD,VIGILANCIA,Wind_AVG,WIND_GUST_10Y,WIND_GUST_30Y,WIND_GUST_50Y,WIND_MAX_10Y,WIND_MAX_30Y,WIND_MAX_50Y,WIND_MEAN_10Y,WIND_MEAN_30Y,WIND_MEAN_50Y,WS_RISK,year,aeropuertos,agricultural_mainland,agua,altura_mea,altura_std,arrozales,Arsenico_invier,Arsenico_otono,Arsenico_primav,Arsenico_verano,Benzo_invier,Benzo_otono,Benz

In [4]:
#Por si llegase a ser necesario reducir el tamaño del df para el analisis
#df_muestra = df_muestra.sample(n=500)


In [5]:


# Calcular la columna 'resp' 
var_resp = 'stro_Corr_AGUAACAGBC'
peso = 'exp_corr_ACAGBC'
ano='year'
df_muestra['resp'] = df_muestra[var_resp] / df_muestra[peso]
valores_distintos = dict['CATEGORÍA'].unique()
variables_categoricas = df_muestra.select_dtypes(include=['object', 'category']).columns
df_muestra = df_muestra.apply(lambda col: col.fillna(col.mean()) if col.dtype in ['float64', 'int64'] else col, axis=0)


In [ ]:
# Llamamos a la función para obtener el DataFrame filtrado y generar el diagrama de correlación
df_filtrado = filtrar_columnas_por_categoria_y_correlacion(df_muestra, dict, valores_distintos, var_resp, peso)
df_filtrado_cols=df_filtrado.sort_values(by='Correlación', ascending=False)
df_filtrado_cols=df_filtrado_cols.head(10)
df_filtrado2 = df_filtrado_cols['Factor'].tolist()
print(df_filtrado2)
# la ejecucion es muy demorada se ejecuta y se dejan quemados los valores resultantes


DataFrame filtrado por la categoría '03.Agente':
Top 5 correlaciones más fuertes para la categoría '03.Agente':
                  Factor  Correlación
13       POLAGE_TURISMOS     0.020768
14  POLAGE_VEH_COMERCIAL     0.016424
6        POLAGE_COMERCIO     0.015737
4      POLAGE_ACCIDENTES     0.015591
10          POLAGE_MOTOS     0.015158
DataFrame filtrado por la categoría '06.Geográfica':


NameError: name 'df_filtrado_cols' is not defined

In [7]:
#ejecutando  todo el codigo
#df_filtrado2.extend(['year','anualidad_seguro','SUPERFICIE','stro_1a_AGUAACAGBC','AnyoCons_edif_total','AUX','bloqueo','resp','CANAL','catage_emp','catage_glob','catage_hog','catage_tur','CATTAG','cl_cobro','tipologia_literal'])
#con valores quemados
df_filtrado2=[]
df_filtrado2.extend(['year','anualidad_seguro','SUPERFICIE','stro_1a_AGUAACAGBC','AnyoCons_edif_total','AUX','bloqueo','resp','CANAL','catage_emp','catage_glob','catage_hog','catage_tur','CATTAG','cl_cobro','tipologia_literal','stro_Corr_AGUAACAGBC', 'exp_corr_ACAGBC', 'CUPD_CAP_Corr_aguaacagbc', 'EXPOSICION', 'K_ACAGBC', 'FP', 'p_servJ', 'cp_ter_peso_2012', 'estu3grado_16_64', 'estu3grado'])

df_muestra=df_muestra[df_filtrado2]
df_muestra.head(5)
#f_muestra = df_muestra.drop(columns=['CODINE', 'CODPOST','COMESP','CONFCOMELE','Etiq_AGUAACAGBC','NIF_TIPO','NP_RB','PACRAS','PROPIETARIO','ROBINSON','TAG','VIGILANCIA'])
#df_muestra.columns


,year,anualidad_seguro,SUPERFICIE,stro_1a_AGUAACAGBC,AnyoCons_edif_total,AUX,bloqueo,resp,CANAL,catage_emp,catage_glob,catage_hog,catage_tur,CATTAG,cl_cobro,tipologia_literal,stro_Corr_AGUAACAGBC,exp_corr_ACAGBC,CUPD_CAP_Corr_aguaacagbc,EXPOSICION,K_ACAGBC,FP,p_servJ,cp_ter_peso_2012,estu3grado_16_64,estu3grado
0,2020.0,3.0,2057.0,0.0,1969.0,4.0,4461.798995,6.969663,C,1,5,9,1,2,Bancario,NIVEL MEDIO PROVINCIAL RESIDENCIAL,0.0,0.000000,0.00,0.748634,0.00,1.0,0.0386,0.794620,0.094087,0.126541
1,2023.0,8.0,700.0,0.0,1976.0,2.0,230.000000,6.969663,C,1,5,3,7,None,Bancario,TURISMO INTERIOR DE 2Âª RESIDENCIA,0.0,0.000000,0.00,0.000000,0.00,1.0,0.0342,0.713630,0.096059,0.096296
2,2021.0,2.0,1020.0,1.0,1971.0,1.0,4461.798995,0.000000,C,1,4,9,2,P,Bancario,ZONAS TURÃSTICAS DEL INTERIOR,0.0,0.665753,0.00,0.665753,50050.00,1.0,0.0364,0.747019,0.190480,0.205109
3,2020.0,8.0,2700.0,1.0,1964.0,7.0,4881.000000,3.000000,D,1,5,2,5,H,Bancario,NIVEL MEDIO PROVINCIAL RESIDENCIAL,1.0,0.333333,48.23,0.333333,41488.54,1.0,0.0487,0.847111,0.251241,0.310950
4,2021.0,2.0,658.0,0.0,1961.0,4.0,4881.000000,0.000000,C,1,9,9,5,2,Bancario,NIVEL MEDIO DE MUNICIPIOS RELEVANTES,0.0,0.832877,0.00,0.832877,70070.00,1.0,0.0174,0.611199,0.094947,0.105901


In [8]:
moda = df_muestra['catage_glob'].mode()[0]
moda

'4'

In [9]:

# Identificar las columnas categóricas en df_muestra
categorical_cols = df_muestra.select_dtypes(include=['object', 'category']).columns
# Reemplazar NaN en todas las columnas categóricas con la moda (valor más frecuente)
for col in categorical_cols:
    moda = df_muestra['catage_glob'].mode()[0]  # Obtener el valor más frecuente (la moda)
    df_muestra[col] = df_muestra[col].fillna(moda)  # Rellenar NaN con la moda
df_muestra = pd.DataFrame(df_muestra)
#df_muestra = df_muestra.T.drop_duplicates().T


In [10]:
categorical_cols = df_muestra.select_dtypes(include=['object', 'category']).columns 
categorical_cols

Index(['CANAL', 'catage_emp', 'catage_glob', 'catage_hog', 'catage_tur',
       'CATTAG', 'cl_cobro', 'tipologia_literal'],
      dtype='object')

In [11]:
factores=df_muestra.columns.tolist()
df_muestra[factores].dtypes

year                        float64
anualidad_seguro            float64
SUPERFICIE                  float64
stro_1a_AGUAACAGBC          float64
AnyoCons_edif_total         float64
AUX                         float64
bloqueo                     float64
resp                        float64
CANAL                        object
catage_emp                   object
catage_glob                  object
catage_hog                   object
catage_tur                   object
CATTAG                       object
cl_cobro                     object
tipologia_literal            object
stro_Corr_AGUAACAGBC        float64
exp_corr_ACAGBC             float64
CUPD_CAP_Corr_aguaacagbc    float64
EXPOSICION                  float64
K_ACAGBC                    float64
FP                          float64
p_servJ                     float64
cp_ter_peso_2012            float64
estu3grado_16_64            float64
estu3grado                  float64
dtype: object

In [12]:
# Creamos variables dummy para las columnas categóricas

key_words = df_muestra.select_dtypes(include=['object', 'category']).columns.tolist()
df_encoded = pd.get_dummies(df_muestra, columns=key_words, drop_first=True)

df_encoded.head(5)

,year,anualidad_seguro,SUPERFICIE,stro_1a_AGUAACAGBC,AnyoCons_edif_total,AUX,bloqueo,resp,stro_Corr_AGUAACAGBC,exp_corr_ACAGBC,CUPD_CAP_Corr_aguaacagbc,EXPOSICION,K_ACAGBC,FP,p_servJ,cp_ter_peso_2012,estu3grado_16_64,estu3grado,CANAL_B,CANAL_C,CANAL_D,CANAL_R,catage_emp_2,catage_emp_3,catage_emp_4,catage_emp_5,catage_emp_6,catage_emp_7,catage_emp_9,catage_glob_2,catage_glob_3,catage_glob_4,catage_glob_5,catage_glob_6,catage_glob_7,catage_glob_9,catage_hog_2,catage_hog_3,catage_hog_4,catage_hog_5,catage_hog_6,catage_hog_7,catage_hog_9,catage_tur_2,catage_tur_3,catage_tur_4,catage_tur_5,catage_tur_6,catage_tur_7,catage_tur_9,CATTAG_3,CATTAG_4,CATTAG_B,CATTAG_E,CATTAG_H,CATTAG_J,CATTAG_K,CATTAG_P,CATTAG_V,cl_cobro_Bancario,cl_cobro_Físico o Ventanilla,tipologia_literal_ACOMODADOS DE MUNICIPIOS RELEVANTES,tipologia_literal_ACOMODADOS PROVINCIALES RESIDENCIALES,tipologia_literal_ACOMODADOS PROVINCIALES ZONA CENTRO,tipologia_literal_ALTO NIVEL PROVINCIAL RESIDENCIAL,tipologia_literal_ALTO NIVEL PROVINCIAL ZONA CENTRO,tipologia_literal_BAJO NIVEL DE MUNICIPIOS RELEVANTES,tipologia_literal_BAJO NIVEL PROVINCIAL RESIDENCIAL,tipologia_literal_CENTROS RURALES,tipologia_literal_COLONIAS EXTRANJERAS,tipologia_literal_NIVEL MEDIO DE MUNICIPIOS RELEVANTES,tipologia_literal_NIVEL MEDIO PROVINCIAL RESIDENCIAL,tipologia_literal_NIVEL MEDIO PROVINCIAL ZONA CENTRO,tipologia_literal_NUEVOS DESARROLLOS-PAUS,tipologia_literal_PEQUEÃOS MUNICIPIOS DE ALTO NIVEL ECONÃMICO,tipologia_literal_PERIFERIA METROPOLITANA ACOMODADA,tipologia_literal_PERIFERIA METROPOLITANA DE ALTO NIVEL,tipologia_literal_PERIFERIA METROPOLITANA DE NIVEL BAJO,tipologia_literal_PERIFERIA METROPOLITANA DE NIVEL MEDIO,tipologia_literal_PERIFERIA METROPOLITANA NIVEL MEDIO DE CINTURONES INDUSTRIALES,tipologia_literal_RURAL AGRÃCOLA/GANADERO,tipologia_literal_RURAL DEPRIMIDO,tipologia_literal_RURAL MANUFACTURERO Y MINERO,tipologia_literal_TURISMO INTERIOR DE 2Âª RESIDENCIA,tipologia_literal_ZONA CENTRO ALTO NIVEL,tipologia_literal_ZONA CENTRO DE NIVEL MEDIO,tipologia_literal_ZONA RESIDENCIAL DE ALTO NIVEL,tipologia_literal_ZONA RESIDENCIAL DE BAJO NIVEL,tipologia_literal_ZONA RESIDENCIAL DE NIVEL MEDIO,tipologia_literal_ZONA URBANA DE ALTO NIVEL,tipologia_literal_ZONA URBANA DE BAJO NIVEL,tipologia_literal_ZONA URBANA DE NIVEL MEDIO,tipologia_literal_ZONAS DE LUJO/ÃLITE,tipologia_literal_ZONAS PESQUERAS,tipologia_literal_ZONAS RESIDENCIALES DE MUNICIPIOS TURÃSTICOS,tipologia_literal_ZONAS TURÃSTICAS DE 2Âª RESIDENCIA,tipologia_literal_ZONAS TURÃSTICAS DEL INTERIOR
0,2020.0,3.0,2057.0,0.0,1969.0,4.0,4461.798995,6.969663,0.0,0.000000,0.00,0.748634,0.00,1.0,0.0386,0.794620,0.094087,0.126541,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2023.0,8.0,700.0,0.0,1976.0,2.0,230.000000,6.969663,0.0,0.000000,0.00,0.000000,0.00,1.0,0.0342,0.713630,0.096059,0.096296,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2021.0,2.0,1020.0,1.0,1971.0,1.0,4461.798995,0.000000,0.0,0.665753,0.00,0.665753,50050.00,1.0,0.0364,0.747019,0.190480,0.205109,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,F

In [13]:
cols_encoded = [each for each in df_encoded.columns if any(keyword in each for keyword in key_words)]
cols_encoded

['CANAL_B',
 'CANAL_C',
 'CANAL_D',
 'CANAL_R',
 'catage_emp_2',
 'catage_emp_3',
 'catage_emp_4',
 'catage_emp_5',
 'catage_emp_6',
 'catage_emp_7',
 'catage_emp_9',
 'catage_glob_2',
 'catage_glob_3',
 'catage_glob_4',
 'catage_glob_5',
 'catage_glob_6',
 'catage_glob_7',
 'catage_glob_9',
 'catage_hog_2',
 'catage_hog_3',
 'catage_hog_4',
 'catage_hog_5',
 'catage_hog_6',
 'catage_hog_7',
 'catage_hog_9',
 'catage_tur_2',
 'catage_tur_3',
 'catage_tur_4',
 'catage_tur_5',
 'catage_tur_6',
 'catage_tur_7',
 'catage_tur_9',
 'CATTAG_3',
 'CATTAG_4',
 'CATTAG_B',
 'CATTAG_E',
 'CATTAG_H',
 'CATTAG_J',
 'CATTAG_K',
 'CATTAG_P',
 'CATTAG_V',
 'cl_cobro_Bancario',
 'cl_cobro_Físico o Ventanilla',
 'tipologia_literal_ACOMODADOS DE MUNICIPIOS RELEVANTES',
 'tipologia_literal_ACOMODADOS PROVINCIALES RESIDENCIALES',
 'tipologia_literal_ACOMODADOS PROVINCIALES ZONA CENTRO',
 'tipologia_literal_ALTO NIVEL PROVINCIAL RESIDENCIAL',
 'tipologia_literal_ALTO NIVEL PROVINCIAL ZONA CENTRO',
 'tipolog

In [14]:
factores_updated=[each for each in factores if each not in key_words]
factores_updated

['year',
 'anualidad_seguro',
 'SUPERFICIE',
 'stro_1a_AGUAACAGBC',
 'AnyoCons_edif_total',
 'AUX',
 'bloqueo',
 'resp',
 'stro_Corr_AGUAACAGBC',
 'exp_corr_ACAGBC',
 'CUPD_CAP_Corr_aguaacagbc',
 'EXPOSICION',
 'K_ACAGBC',
 'FP',
 'p_servJ',
 'cp_ter_peso_2012',
 'estu3grado_16_64',
 'estu3grado']

In [17]:
# Prepara los datos
X = df_encoded[list(set(cols_encoded+factores_updated))].astype('float').fillna(0)
X = sm.add_constant(X)            # Agrega una constante al modelo (intercepto)

y = df_muestra['resp']                   # Variable de respuesta

# Definir la exposición como un offset en logaritmo

offset = np.log(df_muestra[peso]).replace(-np.inf, 0).replace([-np.inf, np.nan], 0)
# Crear el modelo GLM con enlace logarítmico y familia Poisson
modelo1 = sm.GLM(y, X, family=sm.families.Poisson(), offset=offset)

# Ajustar el modelo
resultado = modelo1.fit()

# Mostrar los resultados del modelo
print(resultado.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   resp   No. Observations:               371390
Model:                            GLM   Df Residuals:                   371292
Model Family:                 Poisson   Df Model:                           97
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -7.0807e+05
Date:                Sat, 30 Nov 2024   Deviance:                   6.0627e+05
Time:                        22:32:42   Pearson chi2:                 1.16e+07
No. Iterations:                     9   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                                                       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

In [19]:

calculate_bic(resultado)

1417401.2345724695

¿Qué significa este valor?
El BIC es una medida de la bondad del ajuste de un modelo, pero penaliza la complejidad del modelo (el número de parámetros). El valor de BIC se utiliza para comparar diferentes modelos:

Un BIC más bajo indica un modelo mejor ajustado, que es menos complejo (en términos de número de parámetros) en relación con los datos.
Un BIC más alto sugiere un modelo más complejo o que no ajusta tan bien los datos.
Comparación de Modelos:
Este valor por sí solo no es suficiente para evaluar la calidad del modelo. Para entender qué significa este valor de BIC, necesitas compararlo con los BIC de otros modelos ajustados a los mismos datos o en el contexto de la misma tarea. El modelo con el BIC más bajo generalmente es considerado el más adecuado.

¿Qué puedes hacer con este valor?
Si estás comparando varios modelos, puedes usar el BIC de cada modelo para decidir cuál tiene el mejor ajuste (es decir, el modelo que balancea mejor el ajuste de los datos y la complejidad).

Si deseas un análisis más profundo de la interpretación del BIC, necesitarías más contexto sobre los modelos que estás comparando y la estructura de los datos.